<a href="https://colab.research.google.com/github/Joongeun/Internship/blob/main/VADER_Sent_Analysis_BertTopic_on_Exp%2BAdvice_fit()%2C_then_transform()%2C_not_fit_transform().ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fit on all posts, then transformed (predicted) on 3 categories (1-7 upvotes, 8 and above upvotes, top 10% upvotes)
also used "reduce_outliers()" function and "seed_topic_list" parameter for BERTopic


In [33]:
import numpy as np
import pandas as pd
import json

In [34]:
exp_posts = pd.read_csv('/content/experience_posts.csv')

In [35]:
scores = list(exp_posts['score'])
# for ind, i in enumerate(sorted(scores)):
#   if ind == 348:
#     low = i
#   if ind == 696:
#     mid = i
#   if ind == 1044:
#     high = i

texts = [[], [], [], [], []]
for ind, score in enumerate(exp_posts['score']):
  if score == 0:
    texts[0].append(exp_posts['selftext'][ind])
  elif score > 0 and score <=7:
    texts[1].append(exp_posts['selftext'][ind])
  # else:
  #   texts[2].append(exp_posts['selftext'][ind]) # 8 or above
  if score >= 8:
    texts[2].append(exp_posts['selftext'][ind])
  if score >= 13:
    texts[3].append(exp_posts['selftext'][ind])
  if score >= 22:
    texts[4].append(exp_posts['selftext'][ind])

print(len(texts[0]), len(texts[1]), len(texts[2]), len(texts[3]), len(texts[4]))

low = {"text": texts[0]}
low_mid = {"text": texts[1]}
high_mid = {"text": texts[2]}
high = {"text": texts[3]}
top_10 = {"text":texts[4]}
low_df = pd.DataFrame(low)
low_mid_df = pd.DataFrame(low_mid)
high_mid_df = pd.DataFrame(high_mid)
high_df = pd.DataFrame(high)
top_10_df = pd.DataFrame(top_10)

35 1786 1513 864 351


In [36]:
!pip install bertopic
from bertopic import BERTopic
import json
import pandas as pd
from google.colab import files

In [42]:
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

stop_words = set(stopwords.words('english'))
new_stopwords =["vaping", "vape", "nicotine", "cigarettes", "cigarette", "juul", "juice", "juices", "smoking",
                "smoke", "tobacco", "nic", "cigs", "vaped", "vapes"]
new_stopwords2 =["im","quit", "quitting", "ive", "i've"]
new_stopwords3 =["year","years", "day", "days", "since", "ago", "week", "weeks", "today", "months", "month"]
# stop_words = stop_words.union(new_stopwords, new_stopwords2) #, new_stopwords3) - include temporal terms

def clean_text(posts):
  cleaned_posts = []
  for sentences in posts:
    split = re.findall(r"[\w']+|[!?]", sentences)
    i = 0
    while i < len(split)-1:
      if split[i] == split[i+1]:
        del split[i+1]
        i-=1
      i+= 1
    # Remove all instances of \n where only the n remains
    split = [i for i in split if i != "n" and i not in stop_words]
    cleaned_posts.append(" ".join(split))
  return cleaned_posts

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
l, l_m, h_m, h, top_10 = map(list, [low_df["text"], low_mid_df["text"], high_mid_df["text"], high_df["text"], top_10_df["text"]])
l, l_m, h_m, h, top_10 = map(clean_text, [l, l_m, h_m, h, top_10])
from sklearn.feature_extraction.text import CountVectorizer
# we add this to remove stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=list(stop_words))

all_texts = exp_posts['selftext'].tolist()

topic_model = BERTopic(vectorizer_model=vectorizer_model,
                       top_n_words=100,
    language='english', calculate_probabilities=True,
    verbose=True,
    seed_topic_list=[['proud'], ['book'], ['weight', 'gain'], ['withdrawal', 'symptoms'], ['family', 'friends'], ['patches', 'gum', 'lozenges', 'toothpicks', 'cold turkey', 'wean'], ["covid", "pandemic", "lockdown"]])
    # seed_topic_list=[['trigger', 'avoid', 'failed', 'ended up', 'caved', 'relapse', 'relapsed'], ['book'], ['weight', 'gain'], ['withdrawal', 'symptoms'], ['family', 'friends'], ['patches', 'gum', 'lozenges', 'toothpicks', 'cold turkey', 'wean'], ["covid", "pandemic", "lockdown"]])
#nr topic, min topic size, n neighbor
topic_model.fit(all_texts)

Batches:   0%|          | 0/105 [00:00<?, ?it/s]

2023-09-05 16:10:19,019 - BERTopic - Transformed documents to Embeddings


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-09-05 16:10:43,380 - BERTopic - Reduced dimensionality
2023-09-05 16:10:43,820 - BERTopic - Clustered reduced embeddings


In [44]:
import json
def get_topic(saved_column, name):
  #print(len(topics)) will print 3397 - "topics" is a list of numbers that labels posts with topics
  topics, probs = topic_model.transform(saved_column)

  # p = {ind: list(i) for ind, i in enumerate(probs)}

  # json_object = json.dumps(p, indent=4)
  # with open(name+"_probabilities.json", "w") as outfile:
  #     outfile.write(json_object)
  print(1)
  new_topics = topic_model.reduce_outliers(saved_column, topics, strategy="c-tf-idf")
  print(2)
  t = {ind:int(i) for ind, i in enumerate(new_topics)}
  json_object = json.dumps(t, indent=4)
  with open(name+"_topics.json", "w") as outfile:
      outfile.write(json_object)
  print(3)
  df = topic_model.get_topic_info()
  d = {i:0 for i in range(-1, len(new_topics))}
  for i in new_topics:
    d[i] += 1
  for i in range(len(df)):
    df.loc[i, "Count"] = d[i-1]
  df.to_csv(name+".csv", index=False)
  print(df)


get_topic(top_10, "top_10%")

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2023-09-05 16:11:32,054 - BERTopic - Reduced dimensionality
2023-09-05 16:11:32,141 - BERTopic - Calculated probabilities with HDBSCAN
2023-09-05 16:11:32,143 - BERTopic - Predicted clusters


1
2
3
    Topic  Count                                               Name  \
0      -1      0                             -1_im_vaping_quit_vape   
1       0    200                              0_im_vaping_vape_quit   
2       1     38                         1_nicotine_cravings_day_im   
3       2     18                              2_vape_quit_im_vaping   
4       3     13                          3_cravings_vaping_im_vape   
5       4      7                              4_im_better_feel_like   
6       5      9                               5_juul_pods_pod_quit   
7       6      6                            6_quit_im_quitting_last   
8       7      6                             7_cravings_day_im_days   
9       8      6                  8_nicotine_patch_cold_cold turkey   
10      9     15                        9_nicotine_quit_like_really   
11     10      4                        10_vaping_lungs_quit_breath   
12     11      5                           11_book_quit_vaping_vape   


In [40]:
# get_topic(h, "top_25%")

In [45]:
get_topic(h_m, "8_above")

Batches:   0%|          | 0/48 [00:00<?, ?it/s]

2023-09-05 16:34:06,954 - BERTopic - Reduced dimensionality
2023-09-05 16:34:07,595 - BERTopic - Calculated probabilities with HDBSCAN
2023-09-05 16:34:07,602 - BERTopic - Predicted clusters


1
2
3
    Topic  Count                                               Name  \
0      -1      0                             -1_im_vaping_quit_vape   
1       0    832                              0_im_vaping_vape_quit   
2       1    145                         1_nicotine_cravings_day_im   
3       2     86                              2_vape_quit_im_vaping   
4       3     73                          3_cravings_vaping_im_vape   
5       4     57                              4_im_better_feel_like   
6       5     35                               5_juul_pods_pod_quit   
7       6     27                            6_quit_im_quitting_last   
8       7     22                             7_cravings_day_im_days   
9       8     46                  8_nicotine_patch_cold_cold turkey   
10      9     36                        9_nicotine_quit_like_really   
11     10     22                        10_vaping_lungs_quit_breath   
12     11     32                           11_book_quit_vaping_vape   


In [46]:
get_topic(l_m, "1-7")

Batches:   0%|          | 0/56 [00:00<?, ?it/s]

2023-09-05 16:36:02,132 - BERTopic - Reduced dimensionality
2023-09-05 16:36:02,866 - BERTopic - Calculated probabilities with HDBSCAN
2023-09-05 16:36:02,872 - BERTopic - Predicted clusters


1
2
3
    Topic  Count                                               Name  \
0      -1      0                             -1_im_vaping_quit_vape   
1       0    908                              0_im_vaping_vape_quit   
2       1    125                         1_nicotine_cravings_day_im   
3       2    102                              2_vape_quit_im_vaping   
4       3     89                          3_cravings_vaping_im_vape   
5       4     97                              4_im_better_feel_like   
6       5     50                               5_juul_pods_pod_quit   
7       6     33                            6_quit_im_quitting_last   
8       7     34                             7_cravings_day_im_days   
9       8     82                  8_nicotine_patch_cold_cold turkey   
10      9     34                        9_nicotine_quit_like_really   
11     10     35                        10_vaping_lungs_quit_breath   
12     11     29                           11_book_quit_vaping_vape   


In [ ]:
# get_topic(l, "zero")

In [ ]:
import pandas as pd
import json
top = texts[4]
mid = texts[2]
low = texts[1]
with open('/content/top_10%_topics.json', 'r') as openfile:
    f1 = json.load(openfile)
with open('/content/8_above_topics.json', 'r') as openfile:
    f2 = json.load(openfile)
with open('/content/1-7_topics.json', 'r') as openfile:
    f3 = json.load(openfile)
p1, p2, p3 = [], [], []
for ind, i in enumerate(f1.values()):
  if i == 12:
    p1.append(top[ind])
for ind, i in enumerate(f2.values()):
  if i == 12:
    p2.append(mid[ind])
for ind, i in enumerate(f3.values()):
  if i == 12:
    p3.append(low[ind])
from pprint import pprint
print(len(p1), len(p2), len(p3))
def p(posts):
  c=1
  for i in posts:
    print(c)
    pprint(i)
    print("_"*20)
    c+=1
  print("="*20)
# p(p1)
# p(p2)
p(p3)